# Notebook 5.5: (Optional) Introduction to Perfusion Bioreactor Modeling & Control Concepts

So far in our bioreactor case studies, we've focused on fed-batch operations, where the goal is typically to maximize product at the end of a finite batch. This notebook provides a brief introduction to an alternative and increasingly important mode of operation: **continuous biomanufacturing using perfusion bioreactors**.

Perfusion systems aim for prolonged, stable operation at high cell densities, continuously feeding fresh media and harvesting product. This offers potential advantages in productivity and product consistency. However, it also presents unique modeling and control challenges.

**Goals of this Notebook:**
1. Briefly introduce the concepts of continuous biomanufacturing and perfusion systems.
2. Define a simplified ODE model for a perfusion bioreactor, including perfusion and bleed rates.
3. Discuss the typical control objectives in perfusion systems (e.g., maintaining steady-state Viable Cell Density - VCD).
4. Simulate the open-loop behavior of the perfusion model under different operating conditions.
5. Conceptually outline how NMPC could be applied to control a perfusion bioreactor.
6. Highlight unique challenges associated with perfusion systems.

## 1. Importing Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from ipywidgets import interact, FloatSlider # For interactive plots

# Optional: for nicer plots
plt.rcParams.update({'font.size': 12, 'figure.figsize': (10, 8)})

## 2. Perfusion Bioreactor Systems: An Overview

**Continuous Biomanufacturing Benefits:**
-   Potentially higher volumetric productivity.
-   Improved product consistency due to steady-state operation.
-   Smaller facility footprint.
-   Easier integration with continuous downstream processing.

**Perfusion Principle:**
In a perfusion bioreactor, cells are typically retained within the reactor (e.g., using a filter like ATF or TFF), while spent medium containing product and waste products is continuously removed (perfusate). Fresh medium is continuously added to replenish nutrients and maintain volume.

**Key Manipulated Variables (Inputs for Control):**
-   **Perfusion Rate ($D_p$ or $F_{perfusate}$):** Rate of media exchange (e.g., Reactor Volumes per Day - RVD, or L/hr). This primarily controls the nutrient and waste environment.
-   **Bleed Rate ($D_b$ or $F_{bleed}$):** Rate at which a portion of the cell-containing culture is removed. This is the primary means to control Viable Cell Density (VCD) and prevent overgrowth, maintaining cells in a productive state.

The total dilution rate of soluble components is $D = D_p + D_b$ (if $F_{in} = F_{perfusate} + F_{bleed}$ and volume is constant).

## 3. Simplified Perfusion Bioreactor Model (ODEs)

We'll adapt our previous fed-batch model. For simplicity, let's assume constant volume $V$. The feed input $F_{in}$ is now the sum of fresh media to match $F_{perfusate} + F_{bleed}$.
Let $D_p = F_{perfusate}/V$ and $D_b = F_{bleed}/V$. Total dilution rate $D = D_p + D_b$.

**States:** $[X_v, S, P]^T$ (Volume $V$ is constant).

1.  **Viable Cell Density ($X_v$) Balance:** (Assuming perfect cell retention for $D_p$)
    $$ \frac{dX_v}{dt} = (\mu - \mu_d) X_v - D_b X_v $$

2.  **Substrate ($S$) Balance:**
    $$ \frac{dS}{dt} = -q_S X_v + D (S_{feed} - S) $$
    (Note: $D$ is used here as substrate is diluted by total media throughput if $F_{in} = F_{perfusate} + F_{bleed}$ and $S_{feed}$ is concentration in $F_{in}$)
    Alternatively, if $F_{in}$ only replaces $F_{perfusate}$: $D_p(S_{feed}-S) - q_S X_v$. We'll use the former definition for $D$.

3.  **Product ($P$) Balance:** (Assuming product is not retained by the filter)
    $$ \frac{dP}{dt} = q_P X_v - D P $$

**Kinetic Rate Expressions ($\mu, \mu_d, q_S, q_P$):** We can use the same kinetic expressions as in Notebook 5.1 (Monod, Luedeking-Piret, etc.).

In [2]:
# Define the Perfusion Bioreactor Model ODE function
def perfusion_bioreactor_ode(t, states, params, Dp_val, Db_val):
    Xv, S, P = states # Unpack current states (Volume V is constant)
    
    # Get parameters (same as fed-batch model)
    mu_max = params['mu_max']
    K_S = params['K_S']
    k_d = params['k_d']
    k_d_S = params['k_d_S']
    K_d_S_coeff = params['K_d_S_coeff']
    Y_XS = params['Y_XS']
    m_S = params['m_S']
    alpha = params['alpha']
    beta = params['beta']
    Y_PS = params['Y_PS']
    S_feed = params['S_feed']
    # V_reactor = params['V_reactor'] # Constant volume assumed in rates Dp, Db

    # Total dilution rate for soluble components
    D_total = Dp_val + Db_val 

    # --- Kinetic Rate Calculations ---
    mu = mu_max * S / (K_S + S + 1e-9)
    mu_d_val = k_d + k_d_S * K_d_S_coeff / (K_d_S_coeff + S + 1e-9)
    q_P = alpha * mu + beta
    if Y_PS > 1e6: q_S = (mu / (Y_XS + 1e-9)) + m_S
    else: q_S = (mu / (Y_XS + 1e-9)) + m_S + (q_P / (Y_PS + 1e-9))

    # --- ODEs ---
    dXv_dt = (mu - mu_d_val) * Xv - Db_val * Xv
    # Assuming S_feed is the concentration in the fresh media being perfused.
    # The D_total here implies F_in = (Dp+Db)*V, and S_feed is conc in F_in.
    dS_dt = -q_S * Xv + D_total * (S_feed - S) 
    dP_dt = q_P * Xv - D_total * P
    
    return [dXv_dt, dS_dt, dP_dt]

# Define a default set of parameters (can be same as fed-batch)
perfusion_params = default_params.copy()
# perfusion_params['V_reactor'] = 1.0 # L, not explicitly used if Dp, Db are rates (1/hr)

nx_perf = 3 # Xv, S, P

NameError: name 'default_params' is not defined

## 4. Control Objectives in Perfusion Systems

Unlike fed-batch, the primary goal is often to reach and maintain a **productive steady state**:
-   **Maintain target VCD ($X_{v,sp}$):** This is usually controlled by adjusting the bleed rate $D_b$. At steady state, $\mu - \mu_d = D_b$.
-   **Maintain target substrate concentration ($S_{sp}$):** Often controlled by the perfusion rate $D_p$, ensuring nutrients are supplied and waste is removed without over-diluting or starving cells.
-   Maximize volumetric productivity ($q_P X_v D_p$ or similar, considering product is in perfusate).
-   Ensure consistent product quality.

## 5. Simulating Open-Loop Perfusion Behavior

Let's see how the system behaves with constant $D_p$ and $D_b$ rates.

In [ ]:
# Simulation settings
t_start_perf = 0
t_end_perf = 30*24 # Simulate for 30 days
t_eval_perf = np.linspace(t_start_perf, t_end_perf, 500)

# Initial conditions for perfusion
Xv0_perf = 5.0  # cells/mL or g/L equivalent (higher starting VCD than batch)
S0_perf = 2.0   # g/L
P0_perf = 0.1   # g/L
initial_states_perf = [Xv0_perf, S0_perf, P0_perf]

# --- Function to run and plot perfusion simulation ---
def run_and_plot_perfusion(Dp_val, Db_val, params, title_prefix="Perfusion"):
    sol = solve_ivp(perfusion_bioreactor_ode, 
                      [t_start_perf, t_end_perf], 
                      initial_states_perf, 
                      args=(params, Dp_val, Db_val),
                      dense_output=True, 
                      t_eval=t_eval_perf,
                      method='LSODA'
                     )
    
    Xv_sim, S_sim, P_sim = sol.y
    t_sim = sol.t
    
    fig, axs = plt.subplots(3, 1, figsize=(10, 12), sharex=True)
    title = f'{title_prefix}: Dp={Dp_val:.3f} (1/hr), Db={Db_val:.4f} (1/hr)'
    fig.suptitle(title, fontsize=16)
    
    axs[0].plot(t_sim/24, Xv_sim, 'b-', label='$X_v$ (g/L)') # Time in days
    axs[0].set_ylabel('$X_v$ (g/L)'); axs[0].grid(True); axs[0].legend()

    axs[1].plot(t_sim/24, S_sim, 'g-', label='$S_{glc}$ (g/L)')
    axs[1].set_ylabel('$S_{glc}$ (g/L)'); axs[1].grid(True); axs[1].legend()
    axs[1].set_ylim(bottom=-0.1)

    axs[2].plot(t_sim/24, P_sim, 'r-', label='$P_{prod}$ (g/L)')
    axs[2].set_ylabel('$P_{prod}$ (g/L)'); axs[2].set_xlabel('Time (days)'); axs[2].grid(True); axs[2].legend()
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()
    
    print(f"Conditions at t={t_sim[-1]/24:.1f} days for {title}:")
    print(f"  Xv = {Xv_sim[-1]:.3f} g/L")
    print(f"  S  = {S_sim[-1]:.3f} g/L")
    print(f"  P  = {P_sim[-1]:.3f} g/L")

#### Simulation 1: Low Bleed, Moderate Perfusion

In [ ]:
# Dp_val: reactor volumes per hour. 1 RVD = 1/24 (1/hr)
Dp_test1 = 1.0/24  # 1 RVD (Reactor Volume per Day)
Db_test1 = 0.001 # Low bleed rate (1/hr)
run_and_plot_perfusion(Dp_test1, Db_test1, perfusion_params)

#### Simulation 2: Higher Bleed, Moderate Perfusion

Try to achieve a steady state VCD. If $\mu_{max} \approx 0.08$ and $\mu_d \approx 0.005$, then net growth could be around $0.075$. So $D_b$ should be less than this. Let's target a $\mu_{net} \approx 0.02$, so $D_b = 0.02$.

In [ ]:
Dp_test2 = 1.5/24  # 1.5 RVD 
Db_test2 = 0.02   # Higher bleed rate (1/hr) aiming for steady state
run_and_plot_perfusion(Dp_test2, Db_test2, perfusion_params)

#### Interactive Exploration of Dp and Db

In [ ]:
interact(run_and_plot_perfusion, 
         Dp_val=FloatSlider(min=0.0, max=4.0/24, step=0.25/24, value=1.0/24, format='.4f', description='Dp (1/hr)'),
         Db_val=FloatSlider(min=0.0, max=0.05, step=0.001, value=0.01, format='.4f', description='Db (1/hr)'),
         params=fixed(perfusion_params) # Keep params fixed for this interaction
        );

## 6. Conceptual NMPC for Perfusion Systems

An NMPC controller for a perfusion system would typically manipulate $D_p(t)$ and $D_b(t)$ to achieve its objectives.

**Example Control Objective: Maintain Target $X_{v,sp}$ and $S_{sp}$**

Minimize $J = \sum_{j=1}^{N_p} \left( Q_X (X_{v,k+j|k} - X_{v,sp})^2 + Q_S (S_{k+j|k} - S_{sp})^2 \right) + \sum_{j=0}^{N_c-1} \left( R_{Dp} (D_{p,k+j|k})^2 + R_{Db} (D_{b,k+j|k})^2 + S_{Dp} (\Delta D_{p,k+j|k})^2 + S_{Db} (\Delta D_{b,k+j|k})^2 \right)$

**Decision Variables for NMPC (over $N_c$ or $N_p$):**
- Sequence of future $D_p$ values.
- Sequence of future $D_b$ values.

**Constraints would include:**
- Min/max $D_p, D_b$.
- Min/max $\Delta D_p, \Delta D_b$.
- Min $S$ (to avoid starvation).
- Max $X_v$ (if there's an upper limit beyond which filter fouls or productivity drops).
- Constraints related to filter pressure (if modeled or correlated to $D_p, X_v$).

The NMPC would use the perfusion ODE model for prediction and solve the resulting NLP at each control interval, similar to the fed-batch NMPC setup in CasADi.

**State Estimation:** Similar to fed-batch, EKF or MHE would be needed to estimate $X_v, S, P$ from available online (e.g., capacitance, off-gas) and infrequent offline measurements.

## 7. Unique Challenges and Considerations for Perfusion Control

*   **Long-term Stability:** Perfusion runs can last for weeks or months. Model stability, parameter adaptation, and sensor drift become critical.
*   **Filter Fouling (for ATF/TFF):** A major operational challenge. The control strategy might need to consider filter health, possibly by adjusting $D_p$ or $X_v$ targets, or by scheduling backwashes.
*   **Complex Interactions:** $D_p$ and $D_b$ have coupled effects on cell density, nutrient environment, and waste accumulation.
*   **Defining Optimal Steady State:** The 'best' $X_{v,sp}$ and $S_{sp}$ might not be known a priori and could be a target for higher-level economic optimization (EMPC for perfusion).
*   **Robustness to Biological Variability:** Cell lines can adapt or change their metabolic behavior over long perfusion runs.

Advanced MPC strategies, potentially combined with adaptive modeling and robust techniques, are well-suited to address these complexities.

## 8. Key Takeaways

*   Perfusion bioreactors enable continuous biomanufacturing, aiming for stable, high-density cell cultures.
*   Control typically involves manipulating perfusion rate ($D_p$) for nutrient/waste environment and bleed rate ($D_b$) for VCD control.
*   The modeling approach is similar to fed-batch but accounts for continuous media exchange and cell bleed while volume is often kept constant.
*   NMPC can be a powerful tool for achieving and maintaining desired steady states (e.g., target VCD, substrate levels) in the face of disturbances and constraints.
*   Long-term stability, filter management, and adapting to biological variability are key challenges in perfusion operations.

This notebook provides a conceptual introduction. A full NMPC implementation for perfusion would follow similar steps to Notebooks 5.2/5.3 using CasADi, but with the perfusion model and different control objectives/inputs. The next part of the series (**Part 6: Data-Driven Modeling for MPC**) will explore how to build models when first-principles understanding is incomplete.